## GIS Preprocessing: Uniform Grid Generation for Site Selection

In [ ]:
import geopandas as gpd
import numpy as np
import os

# --- PARAMETERS ---
# WARNING: Update this path to your Fogo Island boundary shapefile
island_boundary_path = "geology_fogo.shp" 

# Grid spacing in meters (adjust as needed)
# 1000m = 1km. Smaller spacing generates more points and longer analysis
grid_spacing_meters = 1000 

# Projected CRS in meters for grid creation (WGS 84 / UTM zone 26N is a good choice)
utm_crs = "EPSG:32626" 

# --- VALIDATION AND PROCESSING ---
if not os.path.exists(island_boundary_path):
    print(f"ERROR: Island boundary file '{island_boundary_path}' not found.")
else:
    try:
        # Load island boundary and reproject to UTM
        island_boundary = gpd.read_file(island_boundary_path)
        island_boundary_utm = island_boundary.to_crs(utm_crs)

        # Get island extent (bounding box) in UTM
        xmin, ymin, xmax, ymax = island_boundary_utm.total_bounds
        print("------------------------------------------------------------------")
        print(f"Study area boundaries for grid generation (in CRS: {utm_crs}):")
        print(f"  X Min (West): {xmin:.2f} m, X Max (East): {xmax:.2f} m")
        print(f"  Y Min (South): {ymin:.2f} m, Y Max (North): {ymax:.2f} m")
        print("------------------------------------------------------------------")
        
        # Create grid points within bounding box
        x_points = np.arange(xmin, xmax, grid_spacing_meters)
        y_points = np.arange(ymin, ymax, grid_spacing_meters)
        grid_points = np.transpose([np.tile(x_points, len(y_points)), np.repeat(y_points, len(x_points))])

        # Convert points to GeoDataFrame
        grid_gdf = gpd.GeoDataFrame(geometry=gpd.points_from_xy(grid_points[:,0], grid_points[:,1]), crs=utm_crs)

        # Keep only points INSIDE the island polygon
        candidate_points_utm = gpd.sjoin(grid_gdf, island_boundary_utm, how="inner", predicate="within")
        candidate_points_utm = candidate_points_utm.drop(columns=['index_right'], errors='ignore')

        # Convert candidate points to lat/lon (EPSG:4326) for analysis functions
        candidate_points_latlon = candidate_points_utm.to_crs("EPSG:4326")

        print(f"\nTotal of {len(candidate_points_latlon)} candidate points generated for analysis.")
        print("\nCandidate points saved in variable 'candidate_points_latlon'.")

    except Exception as e:
        print(f"An error occurred during point generation: {e}")

---
## Spatial Grid Generation: Systematic Sampling for Site Selection

In [ ]:
import geopandas as gpd
import numpy as np
import os
from shapely.geometry import box
import folium

# --- PARAMETERS ---
# WARNING: Update this path to your Fogo Island boundary shapefile
island_boundary_path = "geology_fogo.shp" 

# Grid spacing in meters (square side length)
grid_spacing_meters = 1000 

# Projected CRS in meters for grid creation (WGS 84 / UTM zone 26N recommended)
utm_crs = "EPSG:32626" 

# Output filenames
output_shp_path = "candidate_grid.shp"
output_html_path = "grid_map.html"

# --- VALIDATION AND PROCESSING ---
if not os.path.exists(island_boundary_path):
    print(f"ERROR: Island boundary file '{island_boundary_path}' not found.")
    print("Please provide the correct path to your study area polygon shapefile.")
else:
    try:
        # Load island boundary and reproject to UTM
        island_boundary = gpd.read_file(island_boundary_path)
        island_boundary_utm = island_boundary.to_crs(utm_crs)

        # Get island extent (bounding box) in UTM
        xmin, ymin, xmax, ymax = island_boundary_utm.total_bounds

        print("------------------------------------------------------------------")
        print(f"Study area boundaries for grid generation (in CRS: {utm_crs}):")
        print(f"  X Min (West): {xmin:.2f} m, X Max (East): {xmax:.2f} m")
        print(f"  Y Min (South): {ymin:.2f} m, Y Max (North): {ymax:.2f} m")
        print("------------------------------------------------------------------")

        # Generate coordinates for lower-left corners of grid squares
        grid_x = np.arange(np.floor(xmin), np.ceil(xmax), grid_spacing_meters)
        grid_y = np.arange(np.floor(ymin), np.ceil(ymax), grid_spacing_meters)
        
        # Create square polygons
        polygons = []
        for x in grid_x:
            for y in grid_y:
                # Create grid_spacing_meters x grid_spacing_meters square
                polygons.append(box(x, y, x + grid_spacing_meters, y + grid_spacing_meters))
        
        # Convert polygons to GeoDataFrame
        grid_gdf = gpd.GeoDataFrame(geometry=polygons, crs=utm_crs)

        # Keep only squares that intersect the island polygon
        # Using 'intersects' instead of 'within' to include edge squares
        final_grid_gdf = gpd.sjoin(grid_gdf, island_boundary_utm, how="inner", predicate="intersects")
        # Remove duplicate columns from sjoin and keep unique geometries
        final_grid_gdf = final_grid_gdf.drop_duplicates(subset=['geometry']).drop(columns=['index_right'], errors='ignore')

        print(f"\nTotal of {len(final_grid_gdf)} squares (candidate areas) generated.")

        # --- SAVE GRID SHAPEFILE ---
        try:
            final_grid_gdf.to_file(output_shp_path, driver='ESRI Shapefile', encoding='utf-8')
            print(f"\nGrid shapefile saved to: '{output_shp_path}'")
        except Exception as e_save:
            print(f"Error saving grid shapefile: {e_save}")

        # --- GENERATE INTERACTIVE HTML MAP ---
        try:
            print("\nGenerating interactive HTML map...")
            # Reproject data to EPSG:4326 (lat/lon) for Folium
            island_boundary_geo = island_boundary_utm.to_crs("EPSG:4326")
            final_grid_gdf_geo = final_grid_gdf.to_crs("EPSG:4326")
            
            # Find map center
            map_center = [island_boundary_geo.geometry.unary_union.centroid.y, island_boundary_geo.geometry.unary_union.centroid.x]
            
            # Create base map
            m = folium.Map(location=map_center, zoom_start=11, tiles="CartoDB positron")

            # Add island boundary to map
            folium.GeoJson(
                island_boundary_geo,
                style_function=lambda x: {'color': 'black', 'weight': 2, 'fillOpacity': 0.1}
            ).add_to(m)
            
            # Add grid squares to map
            folium.GeoJson(
                final_grid_gdf_geo,
                style_function=lambda x: {'color': 'blue', 'weight': 1, 'fillColor': 'blue', 'fillOpacity': 0.3},
                tooltip=folium.features.GeoJsonTooltip(fields=[]) # Empty tooltip to hide attributes
            ).add_to(m)
            
            # Calculate centroids (center points) of each square
            centroid_points_utm = final_grid_gdf.copy()
            centroid_points_utm['geometry'] = centroid_points_utm.geometry.centroid
            
            # Convert centroids to lat/lon for analysis functions
            candidate_points_latlon = centroid_points_utm.to_crs("EPSG:4326")
            
            print(f"\nTotal of {len(candidate_points_latlon)} square centroid points generated for analysis.")
            
            # Now the 'candidate_points_latlon' variable contains the 473 centroid points
            # ready for use in your criteria analysis script.
            # The rest of the workflow (analysis loop and CSV saving) can continue from here.

            # Save map to HTML file
            m.save(output_html_path)
            print(f"\nInteractive map saved to: '{output_html_path}'")
            print("Open this file in your browser to view the grid over the island.")

        except Exception as e_map:
            print(f"Error generating interactive map: {e_map}")
            
    except Exception as e:
        print(f"An error occurred during processing: {e}")

---
## Spatial Grid Generation: Categorized Candidate Sites

In [ ]:
import geopandas as gpd
import numpy as np
import os
from shapely.geometry import box, Polygon
import folium
from branca.element import Template, MacroElement

# --- PARAMETERS ---
island_boundary_path = "geology_fogo.shp"
grid_spacing_meters = 1000
utm_crs = "EPSG:32626"
geo_crs = "EPSG:4326"  # Latitude/Longitude
output_shp_path = "candidate_grid.shp"
output_html_path = "interactive_grid_map.html"

# --- VALIDATION AND PROCESSING ---
if not os.path.exists(island_boundary_path):
    print(f"ERROR: Island boundary file '{island_boundary_path}' not found.")
else:
    try:
        # Load study area shapefile
        island_boundary = gpd.read_file(island_boundary_path)
        island_boundary_utm = island_boundary.to_crs(utm_crs)  # Convert to UTM CRS
        xmin, ymin, xmax, ymax = island_boundary_utm.total_bounds

        print("------------------------------------------------------------------")
        print(f"Study area boundaries for grid generation (in CRS: {utm_crs}):")
        print(f"  X Min (West): {xmin:.2f} m, X Max (East): {xmax:.2f} m")
        print(f"  Y Min (South): {ymin:.2f} m, Y Max (North): {ymax:.2f} m")
        print("------------------------------------------------------------------")

        # Create grid of squares
        grid_x = np.arange(np.floor(xmin), np.ceil(xmax), grid_spacing_meters)
        grid_y = np.arange(np.floor(ymin), np.ceil(ymax), grid_spacing_meters)

        polygons = []
        for x in grid_x:
            for y in grid_y:
                polygons.append(box(x, y, x + grid_spacing_meters, y + grid_spacing_meters))

        grid_gdf = gpd.GeoDataFrame(geometry=polygons, crs=utm_crs)
        final_grid_gdf = gpd.sjoin(grid_gdf, island_boundary_utm, how="inner", predicate="intersects")
        final_grid_gdf = final_grid_gdf.drop_duplicates(subset=['geometry']).drop(columns=['index_right'], errors='ignore')

        print(f"\nTotal of {len(final_grid_gdf)} squares (candidate areas) generated.")

        # --- SQUARE CATEGORIZATION ---
        # Coordinates for each category
        low_cost_coords = [(14.9253, -24.3791), (14.9342, -24.3697), (14.9339, -24.3418)]
        low_impact_coords = [(14.8264, -24.4174), (14.8354, -24.4173), (14.8621, -24.3799)]
        high_safety_coords = [(14.8171, -24.3990), (14.8352, -24.3987), (14.9978, -24.3968)]
        best_compromise_coords = [(14.8352, -24.3987)]

        # Convert grid to WGS84 (latitude/longitude) for comparison
        final_grid_gdf_geo = final_grid_gdf.to_crs(geo_crs)

        # Add 'categories' column to store multiple categories
        final_grid_gdf_geo['categories'] = [[] for _ in range(len(final_grid_gdf_geo))]

        # Function to check if centroid is near specific coordinates
        def check_category(row, coords, category):
            centroid = row.geometry.centroid
            for coord in coords:
                if np.isclose(centroid.y, coord[0], atol=0.002) and np.isclose(centroid.x, coord[1], atol=0.002):
                    row.categories.append(category)

        # Assign categories based on coordinates
        final_grid_gdf_geo.apply(check_category, axis=1, coords=low_cost_coords, category='low_cost')
        final_grid_gdf_geo.apply(check_category, axis=1, coords=low_impact_coords, category='low_impact')
        final_grid_gdf_geo.apply(check_category, axis=1, coords=high_safety_coords, category='high_safety')
        final_grid_gdf_geo.apply(check_category, axis=1, coords=best_compromise_coords, category='best_compromise')

        # --- GENERATE INTERACTIVE HTML MAP ---
        print("\nGenerating interactive HTML map...")
        island_boundary_geo = island_boundary_utm.to_crs(geo_crs)  # Convert to WGS84
        map_center = [island_boundary_geo.geometry.union_all().centroid.y, island_boundary_geo.geometry.union_all().centroid.x]
        m = folium.Map(location=map_center, zoom_start=11, tiles="CartoDB positron")

        # Add island boundary to map
        folium.GeoJson(
            island_boundary_geo,
            style_function=lambda x: {'color': 'black', 'weight': 2, 'fillOpacity': 0.1}
        ).add_to(m)

        # Define colors for each category
        colors = {
            'low_cost': '#1f77b4',      # blue
            'low_impact': '#2ca02c',    # green
            'high_safety': '#d62728',   # red
            'best_compromise': '#ff7f0e',  # orange
            'normal': '#999999'         # gray
        }

        # Add grid squares to map
        for _, row in final_grid_gdf_geo.iterrows():
            if len(row.categories) == 1:
                # Single category - color whole square
                folium.GeoJson(
                    row.geometry,
                    style_function=lambda feature, color=colors[row.categories[0]]: {
                        'fillColor': color,
                        'color': 'black',
                        'weight': 1,
                        'fillOpacity': 0.5,
                    },
                    tooltip=f"Category: {row.categories[0]}"
                ).add_to(m)
            elif len(row.categories) > 1:
                # Multiple categories - split square into equal parts
                bounds = row.geometry.bounds
                x_min, y_min, x_max, y_max = bounds
                width = (x_max - x_min) / len(row.categories)
                for i, category in enumerate(row.categories):
                    sub_square = Polygon([
                        (x_min + i * width, y_min),
                        (x_min + (i + 1) * width, y_min),
                        (x_min + (i + 1) * width, y_max),
                        (x_min + i * width, y_max),
                        (x_min + i * width, y_min)
                    ])
                    folium.GeoJson(
                        sub_square,
                        style_function=lambda feature, color=colors[category]: {
                            'fillColor': color,
                            'color': 'black',
                            'weight': 1,
                            'fillOpacity': 0.5,
                        },
                        tooltip=f"Category: {category}"
                    ).add_to(m)
            else:
                # No category - mark as normal
                folium.GeoJson(
                    row.geometry,
                    style_function=lambda feature: {
                        'fillColor': colors['normal'],
                        'color': 'black',
                        'weight': 1,
                        'fillOpacity': 0.5,
                    },
                    tooltip="Category: Normal"
                ).add_to(m)

        # Custom legend
        legend_html = """
        {% macro html(this, kwargs) %}
        <div style='position: fixed; 
                    bottom: 50px; left: 50px; width: 200px; height: 160px; 
                    z-index:9999; font-size:14px;
                    background-color: white;
                    border:2px solid grey;
                    padding: 10px;'>
            <b>Legend</b><br>
            <i style='background:#1f77b4; width:10px; height:10px; display:inline-block;'></i> Low Cost<br>
            <i style='background:#2ca02c; width:10px; height:10px; display:inline-block;'></i> Low Impact<br>
            <i style='background:#d62728; width:10px; height:10px; display:inline-block;'></i> High Safety<br>
            <i style='background:#ff7f0e; width:10px; height:10px; display:inline-block;'></i> Best Compromise<br>
            <i style='background:#999999; width:10px; height:10px; display:inline-block;'></i> Normal<br>
        </div>
        {% endmacro %}
        """
        legend = MacroElement()
        legend._template = Template(legend_html)
        m.get_root().add_child(legend)

        # Save map as HTML file
        m.save(output_html_path)
        print(f"\nInteractive map saved to: '{output_html_path}'")
        print("Open this file in your browser to view the grid over the island.")

    except Exception as e:
        print(f"An error occurred during processing: {e}")